In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid")

import warnings 
warnings.filterwarnings('ignore')

my_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [3]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

# Cálculo del St

In [5]:
training_set = auctions
training_set.sort_values(ascending=True,by='date',inplace=True)

In [6]:
#Separamos el set de datos en "ventanas" de 3 días cada una.
ventana1 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-18')) & (training_set['date'] < pd.to_datetime('2019-04-21'))]
ventana2 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-19')) & (training_set['date'] < pd.to_datetime('2019-04-22'))]
ventana3 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-20')) & (training_set['date'] < pd.to_datetime('2019-04-23'))]
ventana4 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-21')) & (training_set['date'] < pd.to_datetime('2019-04-24'))]
ventana5 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-22')) & (training_set['date'] < pd.to_datetime('2019-04-25'))]
ventana6 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-23')) & (training_set['date'] < pd.to_datetime('2019-04-26'))]
ventana7 = training_set.loc[(training_set['date'] > pd.to_datetime('2019-04-24')) & (training_set['date'] < pd.to_datetime('2019-04-27'))]

In [7]:
fecha_iv1 = pd.to_datetime('2019-04-18')
fecha_iv2 = pd.to_datetime('2019-04-19')
fecha_iv3 = pd.to_datetime('2019-04-20')
fecha_iv4 = pd.to_datetime('2019-04-21')
fecha_iv5 = pd.to_datetime('2019-04-22')
fecha_iv6 = pd.to_datetime('2019-04-23')
fecha_iv7 = pd.to_datetime('2019-04-24')

ventana1['st'] = (ventana1.groupby('device_id')['date'].transform(min) -  fecha_iv1).dt.total_seconds()
ventana2['st'] = (ventana2.groupby('device_id')['date'].transform(min) -  fecha_iv2).dt.total_seconds()
ventana3['st'] = (ventana3.groupby('device_id')['date'].transform(min) -  fecha_iv3).dt.total_seconds()
ventana4['st'] = (ventana4.groupby('device_id')['date'].transform(min) -  fecha_iv4).dt.total_seconds()
ventana5['st'] = (ventana5.groupby('device_id')['date'].transform(min) -  fecha_iv5).dt.total_seconds()
ventana6['st'] = (ventana6.groupby('device_id')['date'].transform(min) -  fecha_iv6).dt.total_seconds()
ventana7['st'] = (ventana7.groupby('device_id')['date'].transform(min) -  fecha_iv7).dt.total_seconds()

In [8]:
label1 = ventana1[['device_id','st']].drop_duplicates()

In [9]:
label2 = ventana2[['device_id','st']].drop_duplicates()

# FEATURES

Creamos features en base a los datasets. Los features van a estar divididos por dispositivo para poder mergearlos en un unico dataframe con features de todos los dataframes.

### Features de Auctions

In [10]:
auctions_v1 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-18')) & (auctions['date'] < pd.to_datetime('2019-04-21'))]
auctions_v2 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-19')) & (auctions['date'] < pd.to_datetime('2019-04-22'))]
auctions_v3 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-20')) & (auctions['date'] < pd.to_datetime('2019-04-23'))]
auctions_v4 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-21')) & (auctions['date'] < pd.to_datetime('2019-04-24'))]
auctions_v5 = auctions.loc[(auctions['date'] > pd.to_datetime('2019-04-22')) & (auctions['date'] < pd.to_datetime('2019-04-25'))]

features_auctions_v1 = auctions_v1.groupby('device_id').agg({'device_id':'size'})
features_auctions_v2 = auctions_v2.groupby('device_id').agg({'device_id':'size'})
features_auctions_v3 = auctions_v3.groupby('device_id').agg({'device_id':'size'})
features_auctions_v4 = auctions_v4.groupby('device_id').agg({'device_id':'size'})
features_auctions_v5 = auctions_v5.groupby('device_id').agg({'device_id':'size'})

In [11]:
features_auctions_v1.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v2.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v3.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v4.columns = ['cantidad_de_subastas_en_ventana']
features_auctions_v5.columns = ['cantidad_de_subastas_en_ventana']

In [12]:
Week_Day = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

for dia in Week_Day:
    subastas_por_dia = auctions[auctions['date'].dt.day_name() == dia].groupby('device_id').agg({'device_id':'size'})
    subastas_por_dia.columns = ['cantidad_eventos_{}'.format(dia)]
    
    features_auctions_v1 = features_auctions_v1.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v2 = features_auctions_v2.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v3 = features_auctions_v3.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v4 = features_auctions_v4.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    features_auctions_v5 = features_auctions_v5.merge(subastas_por_dia, how='outer', left_index=True, right_index=True)
    
features_auctions_v1 = features_auctions_v1.fillna(0)
features_auctions_v2 = features_auctions_v2.fillna(0)
features_auctions_v3 = features_auctions_v3.fillna(0)
features_auctions_v4 = features_auctions_v4.fillna(0)
features_auctions_v5 = features_auctions_v5.fillna(0)

### Features de Events

In [16]:
events_v1 = events.loc[(events['date'] >= pd.to_datetime('2019-04-18')) & (events['date'] <= pd.to_datetime('2019-04-21'))]
events_v2 = events.loc[(events['date'] >= pd.to_datetime('2019-04-19')) & (events['date'] <= pd.to_datetime('2019-04-22'))]
events_v3 = events.loc[(events['date'] >= pd.to_datetime('2019-04-20')) & (events['date'] <= pd.to_datetime('2019-04-23'))]
events_v4 = events.loc[(events['date'] >= pd.to_datetime('2019-04-21')) & (events['date'] <= pd.to_datetime('2019-04-24'))]

ventanas_events = [events_v1, events_v2, events_v3, events_v4]

In [17]:
features_events = []

for ventana in ventanas_events:
    features_ventana = ventana.groupby('ref_hash').agg({'ref_hash':'size'})
    features_ventana.columns = ['cantidad_de_eventos_en_ventana']
    features_events.append(features_ventana)

In [18]:
for index, ventana in enumerate(ventanas_events):
    for dia in Week_Day:
        eventos_por_dia = ventana[ventana['date'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
        eventos_por_dia.columns = ['cantidad_eventos_{}'.format(dia)]
        
        features_events[index] = features_events[index].merge(eventos_por_dia, how='outer', left_index=True, right_index=True)
        features_events[index].fillna(0, inplace=True)

#### Eventos por hora

In [19]:
for index, ventana in enumerate(ventanas_events):
    for hora in range(24):
        eventos_por_hora = ventana[ventana['date'].dt.hour == hora].groupby('ref_hash').agg({'ref_hash':'size'})
        eventos_por_hora.columns = ['cantidad_eventos_hora_{}'.format(hora)]
        features_events[index] = features_events[index].merge(eventos_por_hora, how='outer', left_index=True, right_index=True) 
        features_events[index]['cantidad_eventos_hora_{}'.format(hora)].fillna(0, inplace=True)

In [35]:
features_events[0].head()

,cantidad_de_eventos_en_ventana,cantidad_eventos_Monday,cantidad_eventos_Tuesday,cantidad_eventos_Wednesday,cantidad_eventos_Thursday,cantidad_eventos_Friday,cantidad_eventos_Saturday,cantidad_eventos_Sunday,cantidad_eventos_hora_0,cantidad_eventos_hora_1,...,cantidad_eventos_hora_14,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23
ref_hash,,,,,,,,,,,,,,,,,,,,,
41863526108385,88,0.0,0.0,0.0,24.0,21.0,43.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0
161514654074162,8,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
186034136943920,13,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360710529886978,3,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
365882020742330,57,0.0,0.0,0.0,0.0,36.0,21.0,0.0,4.0,6.0,...,1.0,0.0,4.0,1.0,8.0,1.0,3.0,0.0,0.0,0.0


### Clicks

In [21]:
#Hay clicks que suceden fuera del período de análisis.
clicks_a_considerar = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-18')) & (clicks['created'] < pd.to_datetime('2019-04-27'))]

In [22]:
clicks_v1 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-18')) & (clicks['created'] < pd.to_datetime('2019-04-21'))]
clicks_v2 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-19')) & (clicks['created'] < pd.to_datetime('2019-04-22'))]
clicks_v3 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-20')) & (clicks['created'] < pd.to_datetime('2019-04-23'))]
clicks_v4 = clicks.loc[(clicks['created'] > pd.to_datetime('2019-04-21')) & (clicks['created'] < pd.to_datetime('2019-04-24'))]

features_clicks_v1 = clicks_v1.groupby('ref_hash').agg({'ref_hash':'size'})
features_clicks_v2 = clicks_v2.groupby('ref_hash').agg({'ref_hash':'size'})
features_clicks_v3 = clicks_v3.groupby('ref_hash').agg({'ref_hash':'size'})
features_clicks_v4 = clicks_v4.groupby('ref_hash').agg({'ref_hash':'size'})

In [23]:
features_clicks_v1.columns = ['cantidad_de_clicks_en_ventana']
features_clicks_v2.columns = ['cantidad_de_clicks_en_ventana']
features_clicks_v3.columns = ['cantidad_de_clicks_en_ventana']
features_clicks_v4.columns = ['cantidad_de_clicks_en_ventana']

In [24]:
for dia in Week_Day:
    clicks_por_dia = clicks_a_considerar[clicks_a_considerar['created'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
    clicks_por_dia.columns = ['cantidad_clicks_{}'.format(dia)]
    
    features_clicks_v1 = features_clicks_v1.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    features_clicks_v2 = features_clicks_v2.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    features_clicks_v3 = features_clicks_v3.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    features_clicks_v4 = features_clicks_v4.merge(clicks_por_dia, how='outer', left_index=True, right_index=True)
    
features_clicks_v1 = features_clicks_v1.fillna(0)
features_clicks_v2 = features_clicks_v2.fillna(0)
features_clicks_v3 = features_clicks_v3.fillna(0)
features_clicks_v4 = features_clicks_v4.fillna(0)

### Installs

In [25]:
installs_v1 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-18')) & (installs['created'] < pd.to_datetime('2019-04-21'))]
installs_v2 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-19')) & (installs['created'] < pd.to_datetime('2019-04-22'))]
installs_v3 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-20')) & (installs['created'] < pd.to_datetime('2019-04-23'))]
installs_v4 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-21')) & (installs['created'] < pd.to_datetime('2019-04-24'))]
installs_v5 = installs.loc[(installs['created'] > pd.to_datetime('2019-04-22')) & (installs['created'] < pd.to_datetime('2019-04-25'))]

features_installs_v1 = installs_v1.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v2 = installs_v2.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v3 = installs_v3.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v4 = installs_v4.groupby('ref_hash').agg({'ref_hash':'size'})
features_installs_v5 = installs_v5.groupby('ref_hash').agg({'ref_hash':'size'})

In [26]:
features_installs_v1.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v2.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v3.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v4.columns = ['cantidad_de_instalaciones_en_ventana']
features_installs_v5.columns = ['cantidad_de_instalaciones_en_ventana']

In [27]:
for dia in Week_Day:
    instalaciones_por_dia = installs[installs['created'].dt.day_name() == dia].groupby('ref_hash').agg({'ref_hash':'size'})
    instalaciones_por_dia.columns = ['cantidad_installs_{}'.format(dia)]
    
    features_installs_v1 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v2 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v3 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v4 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    features_installs_v5 = features_installs_v1.merge(instalaciones_por_dia, how='outer', left_index=True, right_index=True)
    
features_installs_v1 = features_installs_v1.fillna(0)
features_installs_v2 = features_installs_v2.fillna(0)
features_installs_v3 = features_installs_v3.fillna(0)
features_installs_v4 = features_installs_v4.fillna(0)
features_installs_v5 = features_installs_v5.fillna(0)

## mergeo todos los features por dispositivo

In [54]:
features_v1 = pd.merge(features_auctions_v1.reset_index(),\
                       features_clicks_v1.reset_index(),\
                       left_on='device_id',right_on='ref_hash',how='inner')

In [55]:
del features_v1['device_id']

In [56]:
features_v1 = pd.merge(features_v1,features_installs_v1.reset_index(),on='ref_hash',how='inner')

In [57]:
features_v1 = pd.merge(features_v1,features_events[0].reset_index(),on='ref_hash',how='inner')

In [58]:
features_v1.shape

(4238, 57)

In [59]:
features_v1 = pd.merge(label1[['device_id']],features_v1,left_on='device_id',right_on='ref_hash',how='inner')

In [60]:
features_v1.shape

(3755, 58)

In [61]:
features_v1.sort_values(by='ref_hash',inplace=True)

In [62]:
del features_v1['device_id']

In [63]:
features_v1.head()

,cantidad_de_subastas_en_ventana,cantidad_eventos_Monday_x,cantidad_eventos_Tuesday_x,cantidad_eventos_Wednesday_x,cantidad_eventos_Thursday_x,cantidad_eventos_Friday_x,cantidad_eventos_Saturday_x,cantidad_eventos_Sunday_x,ref_hash,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_14,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23
1471,8.0,0.0,13.0,4.0,6.0,7.0,2.0,2.0,7164788605058735,0.0,...,1.0,0.0,5.0,4.0,0.0,0.0,3.0,4.0,1.0,7.0
2684,158.0,0.0,0.0,0.0,25.0,105.0,28.0,0.0,11447172392196885,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0
2529,4.0,1.0,0.0,73.0,8.0,2.0,0.0,0.0,17411113585415081,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0
964,254.0,33.0,11.0,0.0,42.0,108.0,104.0,39.0,18336104939494418,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1362,6.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,22109383369151163,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
features_v2 = pd.merge(features_auctions_v2.reset_index(),\
                       features_clicks_v2.reset_index(),\
                       left_on='device_id',right_on='ref_hash',how='inner')

In [70]:
del features_v2['device_id']

In [71]:
features_v2 = pd.merge(features_v2,features_installs_v2.reset_index(),on='ref_hash',how='inner')

In [72]:
features_v2 = pd.merge(features_v2,features_events[1].reset_index(),on='ref_hash',how='inner')

In [73]:
features_v2.shape

(4320, 58)

In [74]:
features_v2 = pd.merge(label2[['device_id']],features_v2,left_on='device_id',right_on='ref_hash',how='inner')

In [75]:
features_v2.shape

(3827, 59)

In [76]:
features_v2.sort_values(by='ref_hash',inplace=True)

In [77]:
del features_v2['device_id']

In [78]:
features_v2.head()

,cantidad_de_subastas_en_ventana,cantidad_eventos_Monday_x,cantidad_eventos_Tuesday_x,cantidad_eventos_Wednesday_x,cantidad_eventos_Thursday_x,cantidad_eventos_Friday_x,cantidad_eventos_Saturday_x,cantidad_eventos_Sunday_x,ref_hash,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_14,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23
218,9.0,0.0,13.0,4.0,6.0,7.0,2.0,2.0,7164788605058735,0.0,...,1.0,0.0,6.0,4.0,0.0,0.0,1.0,5.0,1.0,6.0
349,133.0,0.0,0.0,0.0,25.0,105.0,28.0,0.0,11447172392196885,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
480,339.0,38.0,25.0,104.0,304.0,195.0,117.0,77.0,12240396390252142,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2762,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,19138953426032582,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0,0.0,0.0
133,358.0,240.0,284.0,162.0,370.0,423.0,34.0,169.0,22893722123239044,1.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# LABEL

In [79]:
label1 = (label1.loc[label1['device_id'].isin(features_v1['ref_hash'])])[['device_id','st']]

In [80]:
label1.shape

(3755, 2)

In [81]:
tiempo_maximo = 259200
label1['binary'] = np.where(label1['st']>=tiempo_maximo,False,True)
label1.sort_values(by='device_id',inplace=True)

In [82]:
label1_b = label1[['binary','st']].to_records(index=False) 

In [83]:
label2 = (label2.loc[label2['device_id'].isin(features_v2['ref_hash'])])[['device_id','st']]

In [84]:
label2.shape

(3827, 2)

In [85]:
tiempo_maximo = 259200
label2['binary'] = np.where(label2['st']>=tiempo_maximo,False,True)
label2.sort_values(by='device_id',inplace=True)

In [86]:
label2_b = label1[['binary','st']].to_records(index=False) 

In [88]:
features_v1.head()

,cantidad_de_subastas_en_ventana,cantidad_eventos_Monday_x,cantidad_eventos_Tuesday_x,cantidad_eventos_Wednesday_x,cantidad_eventos_Thursday_x,cantidad_eventos_Friday_x,cantidad_eventos_Saturday_x,cantidad_eventos_Sunday_x,ref_hash,cantidad_de_clicks_en_ventana,...,cantidad_eventos_hora_14,cantidad_eventos_hora_15,cantidad_eventos_hora_16,cantidad_eventos_hora_17,cantidad_eventos_hora_18,cantidad_eventos_hora_19,cantidad_eventos_hora_20,cantidad_eventos_hora_21,cantidad_eventos_hora_22,cantidad_eventos_hora_23
1471,8.0,0.0,13.0,4.0,6.0,7.0,2.0,2.0,7164788605058735,0.0,...,1.0,0.0,5.0,4.0,0.0,0.0,3.0,4.0,1.0,7.0
2684,158.0,0.0,0.0,0.0,25.0,105.0,28.0,0.0,11447172392196885,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0
2529,4.0,1.0,0.0,73.0,8.0,2.0,0.0,0.0,17411113585415081,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0
964,254.0,33.0,11.0,0.0,42.0,108.0,104.0,39.0,18336104939494418,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1362,6.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,22109383369151163,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Algoritmo

In [87]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

estimator = CoxPHSurvivalAnalysis()
estimator.fit(features_v1,label1_b)

LinAlgError: Matrix is singular.

In [103]:
pd.Series(estimator.coef_, index=features_v1.columns)

ref_hash                     -8.357441e-22
cantidad_de_eventos           1.150665e-05
cantidad_eventos_Monday      -3.380476e-04
cantidad_eventos_Tuesday      4.059600e-04
cantidad_eventos_Wednesday   -5.670885e-04
cantidad_eventos_Thursday     2.586518e-04
cantidad_eventos_Friday       6.406885e-05
cantidad_eventos_Saturday     3.038192e-04
cantidad_eventos_Sunday       1.380709e-04
dtype: float64

#### El resultado es un vector de coeficientes, uno para cada variable, donde cada valor corresponde a la relación del registro y evento

In [104]:
prediction = estimator.predict(features_v2)

In [105]:
from sksurv.metrics import concordance_index_censored
result = concordance_index_censored(label2['binary'],label2['st'], prediction)
result[0]

0.5021970863822909

# KNN

In [107]:
set_training = pd.merge(train_st[['device_id','st_v1']],df_features, how='inner', left_on='device_id',right_on='ref_hash')

In [108]:
del set_training['device_id']

In [109]:
columns = list(set_training.columns)
columns.remove('st_v1')

In [110]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
X_train, X_test, Y_train, Y_test = train_test_split(set_training.loc[:, columns],
                                                    set_training['st_v1'],
                                                    test_size=0.2,
                                                    random_state=43)

In [111]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )
#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

In [112]:
neigh.fit(X_train,Y_train) #Fit the model using X as training data and Y as target values

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='distance')

In [113]:
predicted = neigh.predict(X_test) #Predict the target for the provided data

In [114]:
neigh.score(X_test, Y_test)

-0.3960931039558926